In [1]:
from fastai.vision import *
from fastai.metrics import *
from numpy.random import randint

In [2]:
labels_file = 'data/train.txt'

def getSpeedList(file):
    with open(file) as f:
        lines = [line.rstrip('\n') for line in f]

    return lines

labels = getSpeedList(labels_file)

In [3]:
path_img = 'vidCaps/'
fnames = get_image_files(path_img)

In [4]:
print(f'Labels: {len(labels)}\nFiles:  {len(fnames)}')

Labels: 20400
Files:  20400


In [32]:
df = pd.DataFrame(columns = ['name', 'label'])
df

,name,label


In [33]:
for i, file in enumerate(fnames):
    speed = labels[i]
    df.loc[i] = [str(file), float(speed)]
#     df[i] = [file, speed]

In [34]:
df.to_csv(r'vidCaps/labels.csv', index=False)

In [35]:
data = pd.read_csv('vidCaps/labels.csv')
data.head()

,name,label
0,vidCaps/28.105569_frame-0.jpg,28.105569
1,vidCaps/28.105569_frame-1.jpg,28.105569
2,vidCaps/28.106527_frame-2.jpg,28.106527
3,vidCaps/28.130404_frame-3.jpg,28.130404
4,vidCaps/28.109243_frame-4.jpg,28.109243


In [36]:
path = 'vidCaps/'

In [37]:
bs = 64
np.random.seed(42)

In [ ]:
data = (ImageItemList.from_folder(path)
        .split_by_valid_func(lambda o: 0.parent.))